## 사전 훈련된 모델(pretrained model) 활용하기

#### 사전 훈련된 모델이란?

- 대규모 이미지 분류 문제를 위해 대량의 데이터셋에서 미리 훈련된 모델
- 원본 데이터셋이 충분히 크고 일반적이라면 사전 훈련된 모델에 의해 학습된 특성의 계층 구조는 실제 일반적인 모델로 효율적인 역할을 할 수 있음
- 새로운 문제가 원래 작업과 완전히 다른 클래스에 대한 것이더라도 많은 컴퓨터 비전 문제에 유용함
    - 예. 대부분 동물이나 생활용품으로 이루어진 ImageNet 데이터셋에 모델을 훈련한 후 모델을 이미지에서 가구 아이템을 식별하는 용도로 사용 가능
    - ImageNet :  1400만개 레이블된 이미지와 1000개 클래스로 구성
- 사전 훈련된 모델 사례
    - VGG16
    - ResNet
    - Inception
    - Xception

#### 사전 훈련된 모델을 사용하는 방법

1. 특성 추출(feature extraction)
2. 미세 조정(fine tuning)

### 사전 훈련된 모델을 사용한 특성 추출

- 특성 추출이란
    사전에 학습된 모델의 표현을 사용하여 새로운 샘플에서 흥미로운 특성을 뽑아내는 것
- 합성곱 기반 층(합성곱과 풀링 층)을 선택하여 새로운 데이터를 통과시키고, 그 출력으로 새로운 분류기를 훈련

#### 합성곱 층만 재사용하는 이유
- 합성곱 층에 의해 학습된 표현이 일반적인어서 재사용 가능
- 컨브넷의 특성 맵은 이미지에 대한 일반적인 컨셉의 존재 여부를 기록한 맵으로 주어진 컴퓨터 비전 문제에 상관없이 유용하게 사용할 수 있음
- 분류기에서 학습한 표현은 모델이 훈련된 클래스 집합에 특화되어 있음
    - 분류기는 전체 사진에 어떤 클래스가 존재할 확률에 관한 정보만 담고 있음
    - 밀집 연결층에서 찾은 표현은 입력 이미지에 있는 객체의 위치정보를 가지고 있지 않음
- 합성곱의 특성 맵은 객체 위치를 고려함
- 특정 합성곱 층에서 추출한 표현의 일반성(과 재사용성) 수준은 모델에 있는 층의 깊이에 달려 있음
    - 모델의 하위 층은 (에지, 색깔, 질감 등) 지역적이고 매우 일반적인 특성 팹을 추출
    - 모델의 상위 층은 ('강아지 눈'이나 '고양이 귀'처럼) 좀 더 추상적인 개념을 추출
- 새로운 데이터 셋이 원본 모델이 훈련한 데이터셋과 많이 다르다면 전체 합성곱 기반 층을 사용하는 것보다는 모델의 하위 층 몇개만 특성 추출에 사용하는 것이 좋음

#### keras.applications 모듈에서 제공하는 이미지 분류 모델들

- Xception
- ResNet
- MobileNet
- EfficientNet
- DenseNet
- MobileNet
- 그 외

#### VGG16 네트워크의 합성곱 기반 층을 사용한 모델

- ImageNet 데이터셋에 훈련된 VGG16 네트워크의 합성곱 기반 층을 사용

**밀집연결층을 놓는 두 가지 방식**
- 방식1
    - 새로운 데이터셋에서 합성곱 기반 층을 실행하고 출력을 넘파이 배열로 디스크에 저장
    - 저장한 데이터를 독립된 밀집 연결 분류기에 입력으로 사용
    - 입력 이미지에 대해 합성곱 기반 층을 한번만 실행하므로 빠르고 비용이 적게 드나 데이터 증식을 할 수 없음
- 방식2
    - 준비한 모델 위에 Dense층을 쌓아 확장
    - 입력데이터에서 엔드-투-앤드로 전체 모델을 실행
    - 모델에 노출된 모든 입력 이미지가 매번 합성곱 기반층을 통과하므로 데이터 증식을 사용할 수 있으나 방식1보다 비용이 많이 듬

#### 데이터 증식을 사용하지 않는 빠른 특성 추출

- 밀집 연결 분류기 정의하고 훈련하기

#### 데이터 증식을 사용한 특성 추출

conv_base와 새로운 밀집 분류기를 연결한 모델 생성 후 입력 데이터를 사용하여 엔드-투-엔드로 실행

**VGG16 합성곱 기반 층 동결(freezing)**
- 층을 동결하는 것은 훈련하는 동안 가중치가 업데이트 되지 않도록 막는 다는 것
- keras에서는 trainable 속성을 False로 설정하여 층이나 모델을 동결
    - 층이나 모델의 훈련 가능한 가중치 리스트가 비게 됨

**새로운 모델 생성**
1. 데이터 증식 단계
2. 동결된 합성곱 기반 츨
3. 밀집 분류기

`이 기법은 연산비용이 크므로 GPU를 사용`

### 사전 훈련된 모델 미세 조정하기

- 모델을 재사용하는 기법 중 하나로서 특성추출을 보완하는 미세조정(fine tuning)
- 미세 조정은 특성 추출에 사용했던 동결 모델의 상위 층 몇개를 동결에서 해제하고 모델에 새로 추가한 츨(밀집연결분류기)과 함께 훈련하는 것
- 주어진 문제에 조금 더 밀접하게 재사용 모델의 표현을 일부 조정하기 때문에 미세조정이라고 부름
- 미세 조정 단계
    1. 사전에 훈련된 기반 네트워크 위에 새로운 네트워크를 추가
    2. 기반 네트워크를 동결
    3. 새로 추가한 네트워크를 훈련
    4. 기반 네트워크에서 일부 층의 동결을 해제(배치 정규화 층은 동결해제하면 안됨)
    5. 동결을 해제한 층과 새로 추가한 층을 함께 훈련

전체 합성곱 기반 층을 미세조정하지 않는 이유
- 합성곱 하위 층들은 좀 더 일반적이고 재사용 가능한 특성들을 인코딩. 새로운 문제에 재활용하도록 수정이 필요한 것은 구체적인 특성이므로 이를 미세조정하는 것이 유리함. 하위 층으로 갈수록 미세조정에 대한 효과가 감소함
- 훈련해야 할 파라미터가 많을수록 과대적합의 위험이 커짐
- 합성곱 기반 층에서 최상위 2~3개 층만 미세조정하는 것이 좋음

모델 미세 조정하기

----